In [7]:
import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
import os

# --- Persiapan Lingkungan Google Colab ---
from google.colab import drive
drive.mount('/content/drive')

# --- PERBAIKAN: Sesuaikan image_size dan class_labels sesuai model yang dilatih ---
# Tentukan path ke model Anda di Google Drive
# Anda HARUS MENGGANTI path ini dengan lokasi model Anda yang sebenarnya di Google Drive.
# Contoh: '/content/drive/My Drive/FolderModelSaya/model_rambut_terlatih.h5'
model_save_path = '/content/drive/MyDrive/hair_type_classifier_model.h5' # <--- GANTI PATH INI SESUAI LOKASI MODEL ANDA

# Sesuaikan dengan ukuran input model yang Anda latih (dari notebook, ini adalah 150x150)
image_size = (150, 150) #

# Sesuaikan dengan label kelas dan URUTANNYA dari train_generator.class_indices di notebook Anda
# Class Indices: {'Straight': 0, 'Wavy': 1, 'curly': 2, 'dreadlocks': 3, 'kinky': 4}
class_labels = ['Straight', 'Wavy', 'curly', 'dreadlocks', 'kinky'] #


# --- Memuat model yang telah disimpan ---
loaded_model = None
try:
    if os.path.exists(model_save_path):
        loaded_model = load_model(model_save_path)
        print("Model berhasil dimuat dari Google Drive.")
    else:
        print(f"Error: File model tidak ditemukan di {model_save_path}. Pastikan path benar dan model sudah diunggah ke Google Drive.")
except Exception as e:
    print(f"Gagal memuat model: {e}. Pastikan model disimpan dalam format yang benar (misal: .h5 atau SavedModel) dan path sudah sesuai.")

# Mapping rekomendasi perawatan untuk setiap tipe rambut
hair_care_recommendations = {
    'curly': "Rambut keriting membutuhkan hidrasi ekstra. Gunakan produk bebas sulfat, kondisioner yang kaya, dan rutin menggunakan masker rambut. Hindari menyisir rambut saat kering dan gunakan teknik 'plop' untuk mengeringkan.",
    'dreadlocks': "Dreadlocks membutuhkan pembersihan rutin untuk menghindari penumpukan produk dan menjaga kebersatan. Gunakan sampo khusus dreadlocks dan pastikan untuk mengeringkan sepenuhnya untuk mencegah jamur. Minyak alami seperti tea tree oil dapat membantu menjaga kesehatan kulit kepala.",
    'kinky': "Rambut kinky sangat kering dan rapuh. Fokus pada pelembapan intensif dengan leave-in conditioner, minyak alami (kelapa, shea butter), dan teknik 'LOC' (Liquid, Oil, Cream). Lindungi rambut di malam hari dengan satin bonnet atau sarung bantal sutra.",
    'Straight': "Rambut lurus cenderung berminyak lebih cepat. Gunakan sampo yang membersihkan tanpa membuat kering dan kondisioner ringan. Hindari produk berlebihan yang bisa membuat rambut lepek. Keramas secara teratur dan sisir rambut dengan lembut.",
    'Wavy': "Rambut bergelombang berada di antara lurus dan keriting. Gunakan produk yang ringan untuk menambah volume dan definisi tanpa membuat rambut berat. Scrunch rambut saat basah untuk membentuk gelombang dan biarkan kering secara alami atau gunakan diffuser."
}

def predict_hair_type(image_path_or_array):
    """
    Melakukan prediksi tipe rambut dari gambar.
    Menerima path gambar atau numpy array gambar (dari Gradio).
    """
    if loaded_model is None:
        return "Model belum dimuat. Tidak dapat melakukan prediksi. Mohon periksa kembali path model Anda di Google Drive."

    try:
        if isinstance(image_path_or_array, str):
            img = Image.open(image_path_or_array).convert('RGB')
        else:
            img = Image.fromarray(image_path_or_array).convert('RGB')

        # Pra-pemrosesan gambar (resize dan normalisasi)
        img = img.resize(image_size) # Menggunakan image_size yang benar
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0) # Tambahkan dimensi batch

        # Melakukan prediksi menggunakan model yang dimuat
        predictions = loaded_model.predict(img_array)[0]

        # Mengurutkan hasil prediksi berdasarkan persentase tertinggi
        sorted_predictions_indices = np.argsort(predictions)[::-1]

        results = {}
        for i in sorted_predictions_indices:
            label = class_labels[i]
            percentage = predictions[i] * 100
            results[label] = percentage

        # Menentukan tipe rambut dengan persentase tertinggi
        predicted_class_index = np.argmax(predictions)
        predicted_label = class_labels[predicted_class_index]

        # Mendapatkan rekomendasi perawatan
        recommendation = hair_care_recommendations.get(predicted_label, "Tidak ada rekomendasi spesifik untuk tipe rambut ini.")

        # Memformat output untuk ditampilkan di Gradio (menggunakan Markdown)
        output_string = "### Hasil Analisis Tipe Rambut:\n\n"
        for label, percentage in results.items():
            output_string += f"- **{label.capitalize()}**: {percentage:.2f}%\n"

        output_string += f"\n---"
        output_string += f"\n### Tipe Rambut Teridentifikasi: **{predicted_label}**" # Menggunakan label asli tanpa capitalize
        output_string += f"\n\n### Rekomendasi Perawatan:\n"
        output_string += recommendation

        return output_string

    except Exception as e:
        return f"Terjadi kesalahan saat memproses gambar atau melakukan prediksi: {e}"


# Membuat antarmuka Gradio
if loaded_model is not None:
    interface = gr.Interface(
        fn=predict_hair_type,
        inputs=gr.Image(type="numpy", label="Unggah Gambar Rambut"),
        outputs=gr.Markdown(label="Hasil Prediksi dan Rekomendasi"),
        title="Klasifikasi Tipe Rambut dan Rekomendasi Perawatan",
        description="Unggah gambar rambut untuk mendapatkan prediksi tipe rambut (Curly, Dreadlocks, Kinky, Straight, Wavy) dan rekomendasi perawatan."
    )

    # Menjalankan antarmuka Gradio
    interface.launch(share=True)
else:
    print("Model belum dimuat. Tidak dapat meluncurkan Gradio interface. Harap pastikan model Anda sudah diunggah ke Google Drive dan path-nya benar.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model berhasil dimuat dari Google Drive.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0323b9ab5b2df920e2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
